In [1]:
import os
import glob
import time
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

# import from py

In [2]:
import sys
sys.path.append("../../")

%run ../../ai/tf/setup/sess.py
%run ../../ai/tf/setup/inits.py
%run ../../ai/tf/setup/hooks.py

%run ../../ai/tf/ops/cells.py
%run ../../ai/tf/ops/convs.py
%run ../../ai/tf/ops/weights.py
%run ../../ai/tf/ops/regularizes.py
%run ../../ai/model/exp/blocks.py
%run ../../ai/dataset/everyone/tfrecord/tfreader.py

# load dataset

In [3]:
tf_src_path = "/home/chy/archive-data/processed/everyone-tfr"
tfrecord_files = glob.glob(os.path.join(tf_src_path, '*.tfr'))

for_train = tfrecord_files[0:150]
for_val = tfrecord_files[150:185]

batch_size = 8
iter_train = to_dataset(for_train, batch_size=batch_size)
frames, lefts, rights, cam_x, cam_y, csx, csy = iter_train.get_next()

iter_val = to_dataset(for_val, batch_size=8)
frames_v, lefts_v, rights_v, cam_x_v, cam_y_v, csx_v, csy_v = iter_val.get_next()

In [4]:
train_item = len(for_train) * 16 * 512
step_per_epoch = int(train_item / batch_size)

# build model

In [5]:
pred_x, pred_y = build_model(frames, lefts, csx, csy, True)

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


# loss

In [6]:
labels = tf.transpose(tf.stack([cam_x, cam_y]))
preds = tf.transpose(tf.stack([pred_x, pred_y]))
mse = tf.losses.mean_squared_error(labels, preds)

loss_x = tf.losses.mean_squared_error(cam_x, pred_x)
loss_y = tf.losses.mean_squared_error(cam_y, pred_y)
loss = loss_x + loss_y

# optimizer

In [7]:
lr_init = 0.001        
decay_steps = 2500
lr_decay_factor = 0.8  

lr = tf.train.exponential_decay(
        learning_rate=lr_init,
        global_step=tf.train.get_or_create_global_step(),
        decay_steps=int(decay_steps),
        decay_rate=lr_decay_factor,
        staircase=True)
    
train_op = tf.train.AdamOptimizer(lr, beta1=0.0, epsilon=1e-3, use_locking=True).minimize(loss)

In [8]:
global_step = tf.train.get_or_create_global_step()
inc_global_step_op = tf.assign(global_step, global_step+1)
init_ops = get_init_ops(iter_train)

In [9]:
hooks = make_hooks("./")

INFO:tensorflow:Create CheckpointSaverHook.


In [11]:
with tf.Session(config=get_sess_config()) as sess:
    sess.run(init_ops)
    
    for i in range(step_per_epoch * 20):
        v_mse, v_x, v_y, _ = sess.run([mse, loss_x, loss_y, train_op])
        
        if i % 50 == 0:
            print("[{:06d}] mse:{:.2f}  ({:.2f},{:.2f})".format(i, v_mse, v_x, v_y))
        
        if i % step_per_epoch == 0:
            print("todo validate")
            
        step = sess.run(inc_global_step_op)

[000000] mse:57.55  (57.05,58.05)
todo validate
[000050] mse:41.26  (28.95,53.57)
[000100] mse:45.95  (82.48,9.41)
[000150] mse:36.71  (27.09,46.33)
[000200] mse:60.01  (42.12,77.90)
[000250] mse:9.71  (11.82,7.60)
[000300] mse:20.46  (33.88,7.04)
[000350] mse:31.24  (6.39,56.10)
[000400] mse:15.04  (18.35,11.74)
[000450] mse:25.30  (34.93,15.67)
[000500] mse:26.96  (45.80,8.12)
[000550] mse:22.12  (27.60,16.63)
[000600] mse:14.32  (15.86,12.78)
[000650] mse:18.68  (23.19,14.16)
[000700] mse:59.64  (47.89,71.39)
[000750] mse:15.27  (21.94,8.59)
[000800] mse:20.59  (23.77,17.41)
[000850] mse:30.66  (34.18,27.14)
[000900] mse:15.17  (13.75,16.60)
[000950] mse:20.11  (24.54,15.68)
[001000] mse:14.25  (9.56,18.94)
[001050] mse:12.38  (15.97,8.80)
[001100] mse:10.88  (9.99,11.78)
[001150] mse:12.46  (10.54,14.37)
[001200] mse:30.62  (12.34,48.90)
[001250] mse:29.99  (13.25,46.72)
[001300] mse:10.57  (12.53,8.61)
[001350] mse:36.42  (14.18,58.66)
[001400] mse:49.33  (49.11,49.56)
[001450] ms

[012600] mse:6.65  (9.92,3.38)
[012650] mse:4.59  (5.68,3.50)
[012700] mse:6.26  (9.40,3.12)
[012750] mse:14.54  (19.14,9.93)
[012800] mse:4.46  (5.39,3.53)
[012850] mse:8.33  (9.11,7.56)
[012900] mse:8.00  (6.29,9.70)
[012950] mse:18.59  (2.41,34.77)
[013000] mse:6.51  (9.90,3.13)
[013050] mse:9.26  (16.88,1.65)
[013100] mse:5.49  (7.88,3.09)
[013150] mse:9.01  (4.94,13.09)
[013200] mse:6.58  (4.84,8.32)
[013250] mse:4.65  (3.25,6.05)
[013300] mse:5.70  (2.70,8.69)
[013350] mse:5.60  (4.15,7.05)
[013400] mse:9.10  (7.52,10.68)
[013450] mse:14.08  (13.13,15.03)
[013500] mse:9.73  (8.04,11.43)
[013550] mse:8.91  (4.34,13.47)
[013600] mse:10.22  (15.19,5.24)
[013650] mse:17.80  (23.37,12.24)
[013700] mse:7.53  (7.40,7.65)
[013750] mse:6.60  (7.61,5.59)
[013800] mse:9.63  (8.37,10.88)
[013850] mse:14.79  (4.55,25.03)
[013900] mse:5.78  (2.71,8.84)
[013950] mse:14.66  (7.17,22.15)
[014000] mse:6.03  (1.31,10.75)
[014050] mse:7.20  (5.62,8.78)
[014100] mse:14.12  (14.98,13.26)
[014150] mse:

[025500] mse:13.45  (6.81,20.09)
[025550] mse:6.21  (4.15,8.28)
[025600] mse:4.16  (6.46,1.85)
[025650] mse:4.11  (7.07,1.14)
[025700] mse:4.12  (6.50,1.73)
[025750] mse:5.88  (6.32,5.43)
[025800] mse:4.36  (5.53,3.18)
[025850] mse:10.73  (14.75,6.71)
[025900] mse:5.76  (5.85,5.68)
[025950] mse:6.99  (7.15,6.83)
[026000] mse:5.99  (8.26,3.73)
[026050] mse:6.10  (6.34,5.86)
[026100] mse:6.00  (3.75,8.26)
[026150] mse:8.16  (8.36,7.95)
[026200] mse:7.99  (13.77,2.21)
[026250] mse:15.05  (20.07,10.03)
[026300] mse:5.43  (2.41,8.46)
[026350] mse:8.23  (7.16,9.30)
[026400] mse:7.14  (5.68,8.61)
[026450] mse:15.51  (18.79,12.23)
[026500] mse:8.29  (12.01,4.57)
[026550] mse:6.27  (7.07,5.46)
[026600] mse:7.49  (3.48,11.49)
[026650] mse:6.75  (7.77,5.73)
[026700] mse:6.83  (3.85,9.81)
[026750] mse:4.25  (3.14,5.37)
[026800] mse:17.50  (15.22,19.77)
[026850] mse:12.64  (2.84,22.43)
[026900] mse:5.09  (4.22,5.95)
[026950] mse:15.61  (12.94,18.29)
[027000] mse:15.06  (7.39,22.73)
[027050] mse:6.1

[038500] mse:8.23  (3.01,13.44)
[038550] mse:2.63  (2.21,3.05)
[038600] mse:15.04  (11.11,18.96)
[038650] mse:10.28  (10.60,9.96)
[038700] mse:3.49  (2.85,4.13)
[038750] mse:6.64  (10.79,2.50)
[038800] mse:8.19  (6.46,9.91)
[038850] mse:5.36  (7.07,3.65)
[038900] mse:5.44  (7.24,3.65)
[038950] mse:4.87  (4.10,5.63)
[039000] mse:13.24  (9.14,17.34)
[039050] mse:5.41  (2.22,8.60)
[039100] mse:4.03  (3.06,5.01)
[039150] mse:4.12  (3.39,4.85)
[039200] mse:6.79  (9.05,4.53)
[039250] mse:25.71  (8.90,42.52)
[039300] mse:6.88  (4.59,9.17)
[039350] mse:7.90  (4.75,11.06)
[039400] mse:5.07  (3.72,6.42)
[039450] mse:5.38  (3.46,7.30)
[039500] mse:3.62  (0.84,6.40)
[039550] mse:9.51  (13.17,5.85)
[039600] mse:9.59  (5.43,13.76)
[039650] mse:8.39  (3.60,13.18)
[039700] mse:5.37  (8.53,2.22)
[039750] mse:7.37  (9.10,5.64)
[039800] mse:13.40  (8.73,18.07)
[039850] mse:9.13  (4.09,14.17)
[039900] mse:3.08  (2.36,3.81)
[039950] mse:16.61  (7.01,26.21)
[040000] mse:7.86  (8.87,6.85)
[040050] mse:4.47  

[051550] mse:6.35  (4.27,8.43)
[051600] mse:8.17  (6.89,9.45)
[051650] mse:2.80  (2.54,3.06)
[051700] mse:3.49  (5.41,1.56)
[051750] mse:9.26  (9.23,9.30)
[051800] mse:2.63  (2.93,2.33)
[051850] mse:11.07  (9.09,13.06)
[051900] mse:5.19  (5.53,4.85)
[051950] mse:12.23  (7.48,16.98)
[052000] mse:4.99  (5.51,4.47)
[052050] mse:4.82  (2.00,7.64)
[052100] mse:8.91  (6.78,11.04)
[052150] mse:7.80  (12.13,3.47)
[052200] mse:7.33  (5.73,8.93)
[052250] mse:6.00  (1.16,10.85)
[052300] mse:6.74  (10.02,3.45)
[052350] mse:5.96  (3.59,8.32)
[052400] mse:2.07  (1.75,2.39)
[052450] mse:3.54  (2.76,4.33)
[052500] mse:3.47  (1.23,5.70)
[052550] mse:8.58  (4.14,13.02)
[052600] mse:4.96  (8.81,1.11)
[052650] mse:12.11  (7.35,16.87)
[052700] mse:2.83  (2.13,3.52)
[052750] mse:11.67  (17.37,5.98)
[052800] mse:6.96  (3.40,10.52)
[052850] mse:3.04  (1.31,4.78)
[052900] mse:7.91  (7.97,7.85)
[052950] mse:9.43  (5.16,13.71)
[053000] mse:7.20  (3.72,10.67)
[053050] mse:9.05  (9.61,8.49)
[053100] mse:6.39  (4.3

[064550] mse:11.06  (12.09,10.04)
[064600] mse:8.03  (7.24,8.83)
[064650] mse:4.80  (1.91,7.68)
[064700] mse:11.21  (11.97,10.45)
[064750] mse:5.97  (8.23,3.70)
[064800] mse:9.37  (13.29,5.46)
[064850] mse:2.75  (3.00,2.51)
[064900] mse:6.53  (4.18,8.88)
[064950] mse:4.90  (5.50,4.31)
[065000] mse:5.96  (3.00,8.93)
[065050] mse:6.06  (2.79,9.33)
[065100] mse:6.03  (4.32,7.73)
[065150] mse:4.96  (5.09,4.84)
[065200] mse:10.50  (14.26,6.75)
[065250] mse:6.75  (2.39,11.11)
[065300] mse:2.49  (3.22,1.77)
[065350] mse:4.18  (5.38,2.97)
[065400] mse:7.30  (3.79,10.81)
[065450] mse:4.63  (3.87,5.39)
[065500] mse:3.27  (3.92,2.62)
[065550] mse:4.50  (3.49,5.52)
[065600] mse:3.41  (4.45,2.38)
[065650] mse:4.85  (7.73,1.98)
[065700] mse:7.81  (8.35,7.27)
[065750] mse:6.05  (5.09,7.01)
[065800] mse:4.07  (5.02,3.12)
[065850] mse:1.64  (2.58,0.70)
[065900] mse:14.30  (16.39,12.22)
[065950] mse:6.27  (5.07,7.48)
[066000] mse:6.10  (7.30,4.90)
[066050] mse:5.30  (3.63,6.97)
[066100] mse:4.63  (4.77,

[077550] mse:5.08  (4.84,5.32)
[077600] mse:5.63  (5.11,6.14)
[077650] mse:2.12  (2.05,2.20)
[077700] mse:8.98  (9.11,8.86)
[077750] mse:4.03  (6.71,1.35)
[077800] mse:3.89  (3.07,4.71)
[077850] mse:5.57  (2.06,9.08)
[077900] mse:2.39  (3.56,1.22)
[077950] mse:1.53  (1.68,1.37)
[078000] mse:2.12  (0.96,3.29)
[078050] mse:5.31  (3.69,6.93)
[078100] mse:5.00  (5.28,4.73)
[078150] mse:12.76  (8.40,17.13)
[078200] mse:7.10  (7.42,6.77)
[078250] mse:5.75  (3.48,8.03)
[078300] mse:7.05  (8.02,6.08)
[078350] mse:10.70  (15.26,6.15)
[078400] mse:7.70  (11.64,3.77)
[078450] mse:21.85  (17.32,26.38)
[078500] mse:7.14  (11.47,2.82)
[078550] mse:8.76  (5.50,12.02)
[078600] mse:15.38  (10.77,19.99)
[078650] mse:10.16  (12.87,7.46)
[078700] mse:9.18  (7.28,11.07)
[078750] mse:4.70  (7.60,1.80)
[078800] mse:21.72  (0.80,42.65)
[078850] mse:7.60  (6.44,8.76)
[078900] mse:7.81  (6.25,9.38)
[078950] mse:23.44  (1.32,45.55)
[079000] mse:1.36  (0.89,1.82)
[079050] mse:7.80  (10.13,5.47)
[079100] mse:5.78 

[090550] mse:6.88  (6.39,7.37)
[090600] mse:4.94  (2.44,7.45)
[090650] mse:8.23  (8.51,7.94)
[090700] mse:5.36  (5.33,5.38)
[090750] mse:8.46  (5.38,11.55)
[090800] mse:11.78  (10.05,13.50)
[090850] mse:4.14  (3.35,4.94)
[090900] mse:5.00  (2.55,7.45)
[090950] mse:6.83  (3.00,10.67)
[091000] mse:4.34  (6.14,2.54)
[091050] mse:10.51  (15.98,5.04)
[091100] mse:4.63  (3.28,5.98)
[091150] mse:5.40  (7.84,2.96)
[091200] mse:9.61  (10.40,8.83)
[091250] mse:3.46  (5.30,1.62)
[091300] mse:8.51  (7.69,9.32)
[091350] mse:6.21  (9.53,2.88)
[091400] mse:18.69  (8.04,29.35)
[091450] mse:8.26  (12.94,3.57)
[091500] mse:4.68  (5.31,4.05)
[091550] mse:11.78  (18.47,5.09)
[091600] mse:9.93  (18.13,1.73)
[091650] mse:5.54  (5.93,5.15)
[091700] mse:5.17  (5.78,4.56)
[091750] mse:3.09  (3.07,3.10)
[091800] mse:3.59  (3.79,3.39)
[091850] mse:8.21  (5.51,10.91)
[091900] mse:6.38  (4.98,7.77)
[091950] mse:1.32  (1.43,1.21)
[092000] mse:2.96  (2.14,3.78)
[092050] mse:6.96  (4.03,9.88)
[092100] mse:6.47  (5.29

[103550] mse:7.30  (2.34,12.26)
[103600] mse:19.86  (27.96,11.75)
[103650] mse:2.12  (2.53,1.72)
[103700] mse:15.45  (7.49,23.42)
[103750] mse:6.51  (8.52,4.50)
[103800] mse:4.07  (4.00,4.14)
[103850] mse:4.20  (4.36,4.03)
[103900] mse:4.92  (1.94,7.91)
[103950] mse:12.99  (19.85,6.13)
[104000] mse:7.89  (10.04,5.73)
[104050] mse:14.52  (25.37,3.67)
[104100] mse:15.31  (5.95,24.66)
[104150] mse:7.01  (6.38,7.65)
[104200] mse:9.32  (5.50,13.15)
[104250] mse:9.58  (12.18,6.97)
[104300] mse:4.60  (6.50,2.70)
[104350] mse:3.44  (3.23,3.65)
[104400] mse:3.50  (4.86,2.13)
[104450] mse:3.63  (5.64,1.63)
[104500] mse:7.30  (13.74,0.86)
[104550] mse:6.57  (1.39,11.76)
[104600] mse:7.18  (10.05,4.31)
[104650] mse:8.54  (8.85,8.23)
[104700] mse:7.93  (8.23,7.64)
[104750] mse:7.35  (11.49,3.22)
[104800] mse:6.85  (3.87,9.82)
[104850] mse:1.05  (1.18,0.92)
[104900] mse:5.76  (4.14,7.39)
[104950] mse:7.74  (4.55,10.94)
[105000] mse:11.47  (7.02,15.92)
[105050] mse:15.12  (5.98,24.27)
[105100] mse:9.

KeyboardInterrupt: 

# TODO
<br>

## TOP priority
- frame과 eye feature concat 전에 각각 dense

<br >

#### everyone
- #item : 1490944
- train step in epoch : 153600 

<br>

### accuracy 
- resize 방식 변경
   - max(h, w)를 기준으로 resize하여 남는 여백을 padding으로 채우는  resize
- ~~데이터셋 iterator val, train 분리 작성~~
- epoch 별 validation 돌리기
- learning rate annealing
- gap 대신 fc 사용
- 필터수 조절
- eye dual channel
- orientation 통제해서 실험
- 카메라로부터 r,Θ 좌표로... 
- frame 비율을 줄여보면서 중요 요소 비중 실험 
- ~~eye에 대한 conv path를 2개 써서 x, y를 분리~~
- ~~마지막 레이어에서 cm 보정 (카메라와 스크린 첫 픽셀)~~
- ~~regularizer, drop out~~
- ~~dataset 개수 카운팅~~

<br>

### opts
- epoch, train time, lr
- check point save

<br>

### th
- loss가 큰 것들의 분포는 어떻게 될까?
   - 
